In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import requests
from IPython.display import display, Markdown


# Load the dataset and initialize required columns
try:
    file_path = "C:/Users/ASUS/Documents/GitHub/github.com/recommendation-system/Dataset/cleaned_dataset.csv"
    df = pd.read_csv(file_path)
    df["Original_Area"] = df["Area"]
    df["Original_Price"] = df["Price"]
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    raise

# Preprocess the data
features = ["Area", "Price"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Function to make recommendations based on KMeans clustering
def recommend_properties_kmeans(price_input, area_input, top_n=5):
    input_df = pd.DataFrame([[area_input, price_input]], columns=["Area", "Price"])
    input_scaled = scaler.transform(input_df)
    input_cluster = kmeans.predict(input_scaled)[0]

    # Filter cluster by constraints
    cluster_data = df[df["Cluster"] == input_cluster].copy()
    cluster_data = cluster_data[
        (cluster_data["Original_Area"] >= area_input) &
        (cluster_data["Original_Price"] <= price_input)
    ]

    if cluster_data.empty:
        return pd.DataFrame()

    cluster_scaled = scaler.transform(cluster_data[["Area", "Price"]])
    distances = euclidean_distances(input_scaled, cluster_scaled)[0]

    cluster_data["Distance"] = distances
    recommended = cluster_data.sort_values("Distance").head(top_n)

    recommended = recommended.rename(columns={
        "Property Type": "Property Type",
        "Location": "Location",
        "District": "District",
        "Bedrooms": "Bedrooms",
        "Bathrooms": "Bathrooms",
        "Original_Area": "Area",
        "Original_Price": "Price",
        "Agency_Name": "Agency"
    })

    headers = ["Property Type", "Location", "District", "Bedrooms", "Bathrooms", "Area", "Price", "Agency"]
    return recommended[headers]

# Template 1: Informative professional Arabic guide
def template_informative(properties):
    explanation = (
        "أنت مساعد عقاري محترف.\n"
        "اعرض التفاصيل باللغة العربية فقط.\n\n"
        "يجب استخدام تسميات الحقول التالية فقط لعرض التفاصيل:\n"
        "نوع العقار، الموقع، الحي، الغرف، دورات المياه، المساحة، السعر، الوكالة\n\n"
        "لا تترجم أو تغير أي من القيم، خصوصًا القيم العربية مثل: شقة، مكتب الاحمدي للعقارات، شرق - قرطبة.\n"
        "احتفظ بها كما هي.\n\n"
        "اعرض كل عقار بالنمط التالي:\n"
        "- نوع العقار: (value)\n"
        "- الموقع: (value)\n"
        "- الحي: (value)\n"
        "- الغرف: (value)\n"
        "- دورات المياه: (value)\n"
        "- المساحة: (value) متر مربع\n"
        "- السعر: (value) ريال\n"
        "- الوكالة: (value)\n\n"
        "بعد عرض كل عقار، قدم فقرة قصيرة باللغة العربية تصف العقار بشكل احترافي، مبينًا لمن هو مناسب وما يميّزه.\n\n"
        "العقارات المقترحة:\n\n"
    )

    for i, prop in enumerate(properties):
        explanation += (
            f"{i+1}.\n"
            f"- نوع العقار: {prop['Property Type']}\n"
            f"- الموقع: {prop['Location']}\n"
            f"- الحي: {prop['District']}\n"
            f"- الغرف: {prop['Bedrooms']}\n"
            f"- دورات المياه: {prop['Bathrooms']}\n"
            f"- المساحة: {prop['Area']} متر مربع\n"
            f"- السعر: {prop['Price']} ريال\n"
            f"- الوكالة: {prop['Agency']}\n\n"
        )

    return explanation

# Template 2: AI-generated friendly Arabic descriptions for all properties
from googletrans import Translator

# Template 2: AI-generated friendly Arabic descriptions for all properties
def template_friendly_descriptive(properties):
    explanations = []
    
    explanations.append("مرحباً بكم في عروضنا العقارية المميزة!\n\n")
    
    translator = Translator()
    
    for i, prop in enumerate(properties):
        prompt = (
            f"اكتب وصفاً عقارياً باللغة العربية فقط (بدون أي كلمات إنجليزية) بأسلوب ودود واحترافي للعقار التالي:\n"
            f"النوع: {prop['Property Type']}\n"
            f"الموقع: {prop['Location']}\n"
            f"الحي: {prop['District']}\n"
            f"الغرف: {prop['Bedrooms']}\n"
            f"الحمامات: {prop['Bathrooms']}\n"
            f"المساحة: {prop['Area']} متر مربع\n"
            f"السعر: {prop['Price']} ريال\n"
            f"الوكالة: {prop['Agency']}\n\n"
            "التعليمات الصارمة:\n"
            "1. يجب أن يكون المحتوى باللغة العربية فقط (ممنوع استخدام أي كلمات إنجليزية)\n"
            "2. ابدأ بجملة ترحيبية\n"
            "3. صف العقار بأسلوب جذاب وسلس\n"
            "4. ركز على المميزات الرئيسية بشكل طبيعي\n"
            "5. اذكر السعر بشكل مقنع\n"
            "6. ساعد العميل على تخيل الحياة في العقار\n"
            "7. اختم بدعوة للتواصل\n"
            "8. استخدم لغة عربية فصيحة ولكن بأسلوب محادثة\n"
            "9. تجنب التكرار والنقاط التعدادية\n"
            "10. اجعل الوصف بين 6-8 جمل\n"
            "11. تأكد من ذكر جميع التفاصيل المهمة\n"
            "12. لا تستخدم علامات التنصيص أو أي رموز غير عربية"
        )
        
        try:
            description = call_llama(prompt)
            description = ''.join([c for c in description if c.isalpha() or c.isspace() or c in '،.؛؟ء-ؤئأةإآةى' or c.isnumeric()])
            # Translate description into English using googletrans
            translated_description = translator.translate(description, src='ar', dest='en').text
            explanations.append(f"العقار رقم {i+1}:\n{translated_description.strip()}\n")
        except Exception as e:
            print(f"Error generating description: {e}")
            explanations.append(
                f"العقار رقم {i+1}:\n"
                f"نوع العقار: {prop['Property Type']}\n"
                f"الموقع: {prop['Location']}\n"
                f"الحي: {prop['District']}\n"
                f"عدد الغرف: {prop['Bedrooms']}\n"
                f"عدد الحمامات: {prop['Bathrooms']}\n"
                f"المساحة: {prop['Area']} متر مربع\n"
                f"السعر: {prop['Price']} ريال\n"
                f"الوكالة: {prop['Agency']}\n\n"
            )
    
    explanations.append(
        "\nنتمنى أن تجد بين هذه العروض ما يناسب أحلامك!\n"
        "لا تتردد في التواصل معنا لترتيب زيارة أو لمعرفة المزيد من التفاصيل.\n"
        "نحن هنا لمساعدتك في العثور على منزل أحلامك."
    )
    
    return "\n".join(explanations)


# Call the LLaMA Chat API (Together API)
def call_llama(prompt, model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.6, top_p=0.85):
    TOGETHER_API_KEY = "tgp_v1_S8njNTMyg7Knr0Gb9UWjLpGOhtcLEpZSM14ywKDAinc"  # Replace with your key

    if not TOGETHER_API_KEY:
        raise ValueError("API key not found.")

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful real estate assistant."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 7000,
        "temperature": temperature,
        "top_p": top_p,
        "repetition_penalty": 1.1,
    }

    try:
        response = requests.post(
            "https://api.together.xyz/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")

# Ensure all columns are unique
def make_columns_unique(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [
            dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))
        ]
    df.columns = cols
    return df

# Generate recommendations and display results
recommended = recommend_properties_kmeans(price_input=2000000, area_input=300)
if recommended.empty:
    print("No properties found based on your input.")
else:
    recommended_unique = make_columns_unique(recommended)
    property_dicts = recommended_unique.to_dict(orient="records")

    # Show only 3 properties
    top3_properties = property_dicts[:3]

    prompt1 = template_informative(top3_properties)
    prompt2 = template_friendly_descriptive(top3_properties)

    try:
        response1 = call_llama(prompt1) if top3_properties else "No properties found"
        response2 = call_llama(prompt2) if top3_properties else "No properties found"

        display(Markdown("### Template 1: Informative Guide"))
        display(Markdown(response1.strip()))

        display(Markdown("### Template 2: Casual Guide"))
        display(Markdown(response2.strip()))
    except Exception as e:
        print("Failed to call API:", str(e))


ReadTimeout: The read operation timed out